In [ ]:
#purchase_log_id, team_id, market_code, price, crt_date from purchase table
#only for active users who signed in within 14days
  
import pandas as pd
df = pd.read_csv("...\\active_purchase_log.txt", sep = "\t")
  
df["new_price"] = df.apply(lambda x: x["price"]*1100 if (x["market_code"] == "APPLE")&(x["purchase_log_id"] < 1456527) else x["price"], axis = 1)
df.drop("price", axis = 1, inplace = True)
df["crt_date"] = pd.to_datetime(df["crt_date"])
  
df_RFM = df.groupby('team_id').agg({'crt_date': lambda x: (df['crt_date'].max().date() - x.max().date()).days, 'purchase_log_id': lambda x: len(x), 'new_price': lambda x: round(x.sum(), 0)})
  
df_RFM.columns = ['Recency', 'Frequency', 'Monetary']
df_RFM = df_RFM.sort_values('Monetary', ascending=False)
quantiles = df_RFM.quantile(q=[0.80])
print(quantiles)
  
import numpy as np
df_RFM['R']=np.where(df_RFM['Recency']<=int(quantiles.Recency.values), 2, 1)
df_RFM['F']=np.where(df_RFM['Frequency']>=int(quantiles.Frequency.values), 2, 1)
df_RFM['M']=np.where(df_RFM['Monetary']>=int(quantiles.Monetary.values), 2, 1)
  
# To do the 2 x 2 matrix we will only use Recency & Monetary
df_RFM['RMScore'] = df_RFM.M.map(str)+df_RFM.R.map(str)
df_RFM = df_RFM.reset_index()
df_RFM_SUM = df_RFM.groupby('RMScore').agg({'team_id': lambda y: len(y.unique()),
                                        'Frequency': lambda y: round(y.mean(),0),
                                        'Recency': lambda y: round(y.mean(),0),
                                        'R': lambda y: round(y.mean(),0),
                                        'M': lambda y: round(y.mean(),0),
                                        'Monetary': lambda y: round(y.mean(),0)})
df_RFM_SUM = df_RFM_SUM.sort_values('RMScore', ascending=False)
df_RFM_SUM
GM
import pandas as pd
df_purchase = pd.read_csv("...\\purchase.txt", sep = "\t")
df_currency = pd.read_csv("...\\currency.txt", sep = "\t")
df_product = pd.read_csv("...\\product.txt", sep = "\t")
  
#currency nan 결측값 존재
df_purchase["currency"] = df_purchase.apply(lambda x: "KRW" if x["market_code"] == "TSTORE" else x["currency"], axis = 1)
df_purchase.dropna(axis = 0, inplace = True)
 
df_1 = pd.merge(df_purchase, df_currency, on = "currency", how = "left")
df_product = df_product[["product_id", "price"]]
df = pd.merge(df_1, df_product, on = "product_id", how = "left")
columns = ["purchase_log_id","product_id","team_id","market_code","currency","price","crt_date","exchange_rate","product_price"]
df.columns = columns
  
df["revenue"] = df.apply(lambda x: x["product_price"] * 1100 if x["price"] == 0 else x["price"]*x["exchange_rate"], axis = 1)
df.drop(["product_id","market_code", "currency","price","exchange_rate","product_price"], axis = 1, inplace = True)
  
df["crt_date"] = pd.to_datetime(df["crt_date"])
df_RFM = df.groupby('team_id').agg({'crt_date': lambda x: (df['crt_date'].max().date() - x.max().date()).days, 'purchase_log_id': lambda x: len(x), 'revenue': lambda x: round(x.sum(), 0)})
df_RFM.columns = ['Recency', 'Frequency', 'Monetary']
df_RFM = df_RFM.sort_values('Monetary', ascending=False)
quantiles = df_RFM.quantile(q=[0.80])
print(quantiles)
  
import numpy as np
df_RFM['R']=np.where(df_RFM['Recency']<=int(quantiles.Recency.values), 2, 1)
df_RFM['F']=np.where(df_RFM['Frequency']>=int(quantiles.Frequency.values), 2, 1)
df_RFM['M']=np.where(df_RFM['Monetary']>=int(quantiles.Monetary.values), 2, 1)
  
# To do the 2 x 2 matrix we will only use Recency & Monetary
df_RFM['RMScore'] = df_RFM.M.map(str)+df_RFM.R.map(str)
df_RFM = df_RFM.reset_index()
df_RFM_SUM = df_RFM.groupby('RMScore').agg({'team_id': lambda y: len(y.unique()),
                                        'Frequency': lambda y: round(y.mean(),0),
                                        'Recency': lambda y: round(y.mean(),0),
                                        'R': lambda y: round(y.mean(),0),
                                        'M': lambda y: round(y.mean(),0),
                                        'Monetary': lambda y: round(y.sum(),0)})
df_RFM_SUM = df_RFM_SUM.sort_values('RMScore', ascending=False)
df_RFM_SUM
  
pd.set_option('display.float_format', lambda x: '%.0f' % x)
df_RFM_SUM